In [1]:
import numpy as np
import scipy
import scipy.signal

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
sns.set(style="white", palette="muted")
%matplotlib inline

In [ ]:
# run cell for interactive plots
import mpld3
mpld3.enable_notebook()

In [2]:
def load_log(filename):
    """Load data from logs with estimated acceleration. Returns
    a numpy record array.
    """
    datatype = np.dtype([
        ('time', '<f8'),
        ('kistler_torque', '<f8'),
        ('motor_torque', '<f8'),
        ('feedback_reference', '<f8'),
        ('estimated_position', '<f8'),
        ('estimated_velocity', '<f8'),
        ('estimated_acceleration', '<f8'),
    ])
    
    values = []
    with open(filename) as f:
        for line in f:
            vals = tuple(float(f) for f in line.split(','))
            assert(len(vals) == len(datatype))
            # check if time is valid,
            # there is usually an invalid time within the first 5 iterations
            if vals[0] > 2**32 - 1:
                assert len(values) <= len(datatype)*5, 'invalid time after {} iterations'.format(
                    len(values)//len(datatype))
                values = []
                continue
            values.append(vals)
    data = np.rec.array(values, dtype=datatype)
    t = data.time
    dt = np.diff(t)
    offset = 2**32 * np.cumsum(dt < 0)
    data.time[1:] += offset
    data.time -= data.time[0]
    data.time /= 168e6 # real time clock frequency
    return data

In [3]:
def plot_log(record, show_plot=True):
    colors = sns.husl_palette(8)
    fig, ax = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
    norm = lambda x: np.abs(x).max()
    normalize = lambda x: x/norm(x)
    ax[0].plot(record.time, normalize(record.estimated_position),
               color=colors[0], label='estimated position/{}'.format(norm(record.estimated_position)))
    ax[0].plot(record.time, normalize(record.estimated_velocity),
               color=colors[1], label='estimated velocity/{}'.format(norm(record.estimated_velocity)))
    ax[0].plot(record.time, normalize(record.estimated_acceleration),
               color=colors[2], label='estimated acceleration/{}'.format(norm(record.estimated_acceleration)))
    ax[0].plot(record.time, record.feedback_reference/norm(record.estimated_velocity),
               color=colors[3], label='feedback velocity/{}'.format(norm(record.estimated_velocity)))
    ax[0].plot(record.time, 0*record.time, color='black', linewidth=1, zorder=1)
    ax[0].legend()
    ax[0].set_xlabel('time [s]')
    ax[0].set_ylabel('[rad, rad/s, rad/s^2]')
    
    inertia = 0.00592455 # kg-m^2
    inertia_torque = inertia*record.estimated_acceleration;
    steer_torque = record.kistler_torque + record.motor_torque + inertia_torque
    ax[1].plot(record.time, record.kistler_torque, color=colors[4], label='sensor torque')
    ax[1].plot(record.time, record.motor_torque, color=colors[5], label='motor torque')
    ax[1].plot(record.time, inertia_torque, color=colors[6], label='inertia torque (calculated)')
    ax[1].plot(record.time, steer_torque, color=colors[7], label='steer torque (calculted)')
    ax[1].legend()
    ax[1].set_xlabel('time [s]')
    ax[1].set_ylabel('torque [N-m]')
    ax[1].plot(record.time, 0*record.time, color='black', linewidth=1, zorder=1)
    if show_plot:
        plt.show()
    return fig, ax

In [4]:
vals = load_log('acc_kalman_u_off.log')
fig, ax = plot_log(vals, False)
fig.suptitle('acc_kalman_u_off.log')
plt.show()

NameError: name 'plot_log2' is not defined

In [ ]:
vals = load_log('acc_kalman_u_on.log')
fig, ax = plot_log(vals, False)
fig.suptitle('acc_kalman_u_on.log')
plt.show()